In [0]:
import TensorFlow

# Modifying machine learning models


In [0]:
struct Model: Layer {
  // TODO: Make me skip connection!
  var conv = Conv2D<Float>(filterShape: (5, 5, 3, 6))
  var maxpool = MaxPool2D<Float>(poolSize: (2, 2), strides: (2, 2))
  @noDerivative var flatten = Flatten<Float>()
  var dense = Dense<Float>(inputSize: 36 * 6, outputSize: 10)

  @differentiable
  func applied(to input: Tensor<Float>, in context: Context) -> Tensor<Float> {
    return input.sequenced(in: context,
                           through: conv, maxpool, flatten, dense)
  }
}


In [0]:
// Use random training data.
let images = Tensor<Float>(randomNormal: [10, 16, 16, 3])
let labels = Tensor<Int32>(rangeFrom: 0, to: 10, stride: 1)

In [0]:
var model = Model()
let opt = SGD(for: model, scalarType: Float.self)
let context = Context(learningPhase: .training)

In [0]:
for i in 0..<10 {
  print("Starting training step \(i)")
  let (loss, gradients) = valueWithGradient(at: model) { model -> Tensor<Float> in
    let logits = model.applied(to: images, in: context)
    return softmaxCrossEntropy(logits: logits, labels: labels)
  }
  print("Loss: \(loss)")
  opt.update(&model.allDifferentiableVariables, along: gradients)
}

# Writing & debugging custom layers


In [0]:
// A custom layer type that's similar to a Dense layer, but with an extra bias
// term.
struct DoubleBiasDense: Layer {
  // TODO(saeta): FILL ME IN!
}


In [0]:
struct Model2: Layer {
  var conv = Conv2D<Float>(filterShape: (5, 5, 3, 6))
  var maxpool = MaxPool2D<Float>(poolSize: (2, 2), strides: (2, 2))
  var flatten = Flatten<Float>()
  var dense = DoubleBiasDense(inputSize: 36 * 6, outputSize: 10)

  @differentiable
  func applied(to input: Tensor<Float>, in context: Context) -> Tensor<Float> {
    return input.sequenced(in: context,
                           through: conv, maxpool, flatten, dense)
  }
}


In [0]:
var model = Model2()
let opt = SGD(for: model, scalarType: Float.self)

for i in 0..<10 {
  print("Starting training step \(i)")
  let (loss, gradients) = valueWithGradient(at: model) { model -> Tensor<Float> in
    let logits = model.applied(to: images, in: context)
    return softmaxCrossEntropy(logits: logits, labels: labels)
  }
  print("Loss: \(loss)")
  opt.update(&model.allDifferentiableVariables, along: gradients)
}

# Experimenting with batch sizes and gradients


In [0]:
for i in 0..<10 {
  print("Starting training step \(i)")
  var grads = Model2.AllDifferentiableVariables.zero
  for j in 0..<4 {
    print("Starting sub-step \(j)")
    let stepGrad = gradient(at: model) { model -> Tensor<Float> in
      let logits = model.applied(to: images, in: context)
      return softmaxCrossEntropy(logits: logits, labels: labels)
    }
    grads += stepGrad
  }
  opt.update(&model.allDifferentiableVariables, along: grads)
}